In [ ]:
import egglib
import glob 
import pandas as pd
import numpy as np
import os
import seaborn as sns

Goal: calculate nucleotide diversity per site and Tajima's D for NLR clades using the egglib python package 

In [ ]:
#gather alignments in fasta format 
directory='/global/scratch/users/chandlersutherland/e14/popgen/popgenome_test'
files = glob.glob(os.path.join(directory, "*.fas"))
#import nlr annotations generated by nlr_annotation.ipynb 
annotations=pd.read_csv('/global/scratch/users/chandlersutherland/e14/popgen/nlr_aa_annotation.csv')[['clade', 'start', 'end', 'domain']]

In [ ]:
#write a function that takes in protein alignment coordinates and clade and outputs the desired stats 
def sub_stat(row):
    #load alignment, and subset by domain
    fasta='/global/scratch/users/chandlersutherland/e14/popgen/popgenome_test/'+row[0]+'.pal2nal.fas'
    clade=row[0]
    domain=row[3]
    aln=egglib.io.from_fasta(fasta, alphabet=egglib.alphabets.DNA)
    codon=egglib.tools.to_codons(aln)
    subset=codon.extract(int(row[1]), int(row[2]))
    
    #compute across subset stats
    dna=egglib.tools.to_bases(subset)
    ls=egglib.stats.ComputeStats()
    ls.configure(multi_hits=True, multi=False)
    ls.add_stats('Pi', 'lseff', 'D')
    stats=ls.process_align(dna, max_missing=0.1)
    
    lseff=stats['lseff']
    D=stats['D']
    Pi=stats['Pi']
    dna_aln=dna.ls
    
    #compute per synonymous and nonsynonomous site stats 
    cs_subset = egglib.stats.CodingDiversity(subset)

    #use the process_sites function to calculate Pi over just the synonymous and non-synonymous sites 
    statsS=ls.process_sites(cs_subset.sites_S)
    statsNS=ls.process_sites(cs_subset.sites_NS) 

    #normalize by the number of synonymous and nonsynonymous sites
    #if there are no polymorphisms at each site, return NaN
    
    if cs_subset.num_pol_S == 0:
        PiS=np.nan
    else:
        PiS=statsS['Pi']/cs_subset.num_sites_S
    
    if cs_subset.num_pol_NS == 0:
        PiNS=np.nan
    else:
        PiNS=statsNS['Pi']/cs_subset.num_sites_NS
    
    return({'clade':clade, 'domain':domain, 'Pi_raw':Pi, 'lseff':lseff, 'dna_aln':dna_aln, 'PiS':PiS, 'PiN':PiNS, 'D':D})

In [ ]:
stats=pd.DataFrame()
#calculate per domain stats 
for i in range(0, len(annotations)):
    row=annotations.iloc[i,:]
    result=sub_stat(row)
    result['start']=row[1]
    result['stop']=row[2]
    #print(result)
    stats=stats.append(result, ignore_index=True)

stats

In [ ]:
#add Col-0 gene names and HV label
gene_table=pd.read_table('/global/scratch/users/chandlersutherland/e14/popgen/Atha_NLRome_GeneTable.txt')[['Gene', 'Clade', 'Ecotype', 'HV']]
gene_table=gene_table[gene_table['Ecotype']=='ATHALIANA']
gene_table['Gene']=gene_table['Gene'].str.split('_', expand=True)[1]
gene_table=gene_table.rename(columns={'Clade':'clade'})[['Gene', 'clade', 'HV']]
gene_table=gene_table.assign(HV=gene_table.HV.map({0:'non-hv', 1:'hv'}))

egglib_results=pd.merge(stats, gene_table)
egglib_results

In [ ]:
egglib_results.to_csv('/global/scratch/users/chandlersutherland/e14/popgen/egglib_results.tsv', sep='\t')

Complement the per domain analysis with sliding window

In [ ]:
cds_only=annotations[annotations['domain']=='cds']

In [ ]:
#function that takes in a clade name, and evaluates Pi, PiS, PiNS, and D across sliding windows of window size 33 codons and step size 10 codons 
#outputs a dataframe with the results across the codon windows 
def my_slider(row, window, step):
    #get codon length 
    codon_length=row[2]
    clade=row[0]
    
    #generate a new annotations dataframe with sliding windows to this length 
    fake_annotations=pd.DataFrame()
    for i in range(0, int(codon_length/step)):
        fake_annotations=fake_annotations.append({'start':0+i*step, 'end':window+i*step, 'clade':clade, 'domain':'window'}, ignore_index=True)
    fake_annotations=fake_annotations[['clade', 'start', 'end', 'domain']]
    return fake_annotations

In [ ]:
#create new annotations df with sliding windows
slide=[]
for i in range(0, len(cds_only)):
    row=cds_only.iloc[i,:]
    windows=my_slider(row, 100, 25)
    slide.append(windows)

slide_df=pd.concat(slide)
slide_df

In [ ]:
window_stats=pd.DataFrame()
for i in range(0, len(slide_df)):
    row=slide_df.iloc[i,:]
    result=sub_stat(row)
    result['start']=row[1]
    result['stop']=row[2]
    #print(result)
    window_stats=window_stats.append(result, ignore_index=True)

window_stats

In [ ]:
vignette=slide_df[slide_df['clade'].isin(['Int8443_258_351_L_88_120_L_38','Int8443_258_351_L_88_120_R_50'])]
for i in range(0, len(vignette)):
    row=slide_df.iloc[i,:]
    result=sub_stat(row)
    result['start']=row[1]
    result['stop']=row[2]
    #print(result)
    window_stats=window_stats.append(result, ignore_index=True)

window_stats

In [ ]:
vignette_window_results=pd.merge(window_stats, gene_table)
vignette_window_results
vignette_window_results.to_csv('/global/scratch/users/chandlersutherland/e14/popgen/egglib_vignette_window_stats.csv')

In [ ]:
egglib_window_final=pd.merge(egglib_window_results, gene_table)
egglib_window_final.to_csv('/global/scratch/users/chandlersutherland/e14/popgen/egglib_window_stats.csv')